In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/머신러닝1/조별활동/와인품질분류/data

/content/drive/MyDrive/Colab Notebooks/머신러닝1/조별활동/와인품질분류/data


In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# 전처리

## label_encoder

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(train['type'])
train['type'] = label_encoder.transform(train['type'])
test['type'] = label_encoder.transform(test['type'])

# 스케일링

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

In [ ]:
minmax_col_list = ['fixed acidity', 'citric acid' ,'chlorides','free sulfur dioxide','density','pH','sulphates']
robust_col_list = ['volatile acidity' ,'total sulfur dioxide', 'residual sugar','alcohol']

In [ ]:
min_max_scaler = MinMaxScaler()
scaled_train_data = min_max_scaler.fit_transform(train[minmax_col_list])
scaled_test_data = min_max_scaler.transform(test[minmax_col_list])

train[minmax_col_list] = scaled_train_data
test[minmax_col_list] = scaled_test_data

In [ ]:
robust_scaler = RobustScaler()
scaled_train_data = robust_scaler.fit_transform(train[robust_col_list])
scaled_test_data = robust_scaler.transform(test[robust_col_list])

train[robust_col_list] = scaled_train_data
test[robust_col_list] = scaled_test_data

# upsampling

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
X = train.drop('quality', axis=1)  # 입력 변수
y = train['quality']  # 타겟 변수

smote = SMOTE(k_neighbors=4)
X_resampled, y_resampled = smote.fit_resample(X, y)


train_resampled = pd.concat([X_resampled, y_resampled], axis=1)
train_resampled

,index,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type,quality
0,0,0.148760,2.382353,0.036145,0.603175,0.054908,0.027778,-0.441558,0.139001,0.551181,0.123596,-0.055556,1,5
1,1,0.413223,1.882353,0.084337,-0.095238,0.096506,0.031250,-0.987013,0.188741,0.354331,0.207865,-0.444444,0,5
2,2,0.338843,-0.470588,0.234940,-0.158730,0.079867,0.069444,0.259740,0.089647,0.244094,0.168539,0.333333,1,5
3,3,0.264463,-0.470588,0.186747,0.476190,0.061564,0.097222,-0.129870,0.130904,0.409449,0.157303,0.277778,1,6
4,4,0.330579,0.647059,0.156627,1.031746,0.083195,0.107639,0.779221,0.161751,0.236220,0.117978,0.333333,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16907,2240,0.305108,0.222912,0.209082,0.287381,0.028276,0.084816,-0.247894,0.099516,0.468574,0.189040,0.770309,1,9
16908,2682,0.363533,0.099231,0.243938,0.790189,0.031674,0.135000,0.051321,0.133805,0.388010,0.111817,0.579008,1,9
16909,2229,0.232282,0.411765,0.175764,-0.207622,0.019790,0.083915,-0.412950,0.049071,0.523938,0.214132,1.172563,1,9
16910,2927,0.254658,0.080756,0.242496,-0.168630,0.030266,0.093538,-0.223948,0.056020,0.509836,0.159036,1.322976,1,9


# 모델링

In [ ]:
y = train_resampled['quality'] #퀄리티 예측해야할거
X = train_resampled.drop('quality', axis=1) #예측해야할거빼고 나머지

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV

In [ ]:
model = AdaBoostClassifier()

stratified_split = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)

n_iter = 0
for train_idx, test_idx in stratified_split.split(X, y):
    X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]

     # 매개변수 그리드 정의
    param = {
     'learning_rate': [0.1, 0.01, 0.001],
      'n_estimators': [100, 500, 1000],
      'learning_rate': [0.1, 0.01, 0.001],
      'n_estimators': [100, 500, 1000],
      'base_estimator': [DecisionTreeClassifier(max_depth=1), DecisionTreeClassifier(max_depth=3)],
      'algorithm': ['SAMME', 'SAMME.R']
    }
    # GridSearchCV 객체 생성
    grid_search = GridSearchCV(model, param_grid=param, cv=5, refit=True, n_jobs=-1, return_train_score=True, verbose=2, error_score='raise')
    # 훈련 데이터로 모델 학습 및 최적 매개변수 탐색
    grid_search.fit(X_train, y_train)
    # 최적 매개변수 출력
    print("최적 매개변수:", grid_search.best_params_)
    # 최적 매개변수로 모델 재학습
    best_model = grid_search.best_estimator_
    best_model.fit(X_train, y_train)

y_pred = best_model.predict(test)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


# 제출파일

In [ ]:
submission = pd.read_csv('sample_submission.csv')

In [ ]:
submission

In [ ]:
submission['quality'] = y_pred

In [ ]:
submission

In [ ]:
submission.to_csv('scale_upsampling_adaboost.csv', index=False)